<a href="https://colab.research.google.com/github/EdBerg21/AI-Professional-Prompts/blob/main/19quantising_jais13b_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Welcome to my notebook!

In this notebook I will load a sharded version of flacon-7b-instruct model in 4-bit, apply double quantisation for optimising memory available in colab GPU and run inference.

Link to sharded model can be found here- https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded

Remember to select: Runtime-> GPU -> T4

Sometimes there is a UTF-8 error in colab. If you get that error- uncomment and run the below command to avert it.

In [ ]:
# Error in colab: NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

Necessary dependencies and imports

In [ ]:
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 4.8 MB/s eta 0:00:00


In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
from torch.utils.data import DataLoader, Dataset

Creating the quantisation config using the bitsandbytes library:

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

Loading the Falcon 7B instruct sharded model:

In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.3 MB/s eta 0:00:00


In [ ]:
model_id = "core42/jais-13b-chat"
model_4bit = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        quantization_config=quantization_config,
        trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin.index.json:   0%|          | 0.00/42.3k [00:00<?, ?B/s]

pytorch_model-00001-of-00006.bin:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

pytorch_model-00002-of-00006.bin:   0%|          | 0.00/9.79G [00:00<?, ?B/s]

pytorch_model-00003-of-00006.bin:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/247 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

Pipeline for text generation

In [ ]:
pipeline = transformers.pipeline(
"text-generation",
model=model_4bit,
tokenizer=tokenizer,
use_cache=True,
device_map="auto",
)

Function for running sequence:

In [ ]:
def your_query(query):
  sequences = pipeline(
      query,
      max_length=200,
      do_sample=True,
      top_k=10,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
  )
  for seq in sequences:
    output = seq['generated_text']
  return output

Output:

In [ ]:
your_query("""Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth.\
 Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the\
 giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:""")

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:569: UserWarning: Some matrices hidden dimension is not a multiple of 64 and efficient inference kernels are not supported for these (slow). Matrix input size found: torch.Size([1, 1, 13653])
  warn(f'Some matrices hidden dimension is not a multiple of {quant_state.blocksize} and efficient inference kernels are not supported for these (slow). Matrix input size found: {A.shape}')


"Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron: Hello, Daniel!\nDaniel: Why are you so obsessed with giraffes?\nGirafatron: Because they're glorious! Look at them go up and down, all graceful and stuff. The way they move is so elegant, I wish I could move like them.\nDaniel: But do you realize other animals have personalities, too? They have feelings!\nGirafatron: But giraffes are glorious! Other animals don't have the same kind of grace and elegance. What about elephants? Elephants are big, but they don't move in the same kind of graceful elegance as the giraffe"